## Hierarchical Forecasting with Different level's of Data

In [68]:
import pandas as pd
import sys
sys.path.insert(0, '../')

import constants as const
import src.utils as util
import src.calculate_errors as err

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [69]:
def calculate_grid_error(model_path):
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    
    results_df = pd.read_csv(f'../{model_path}/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    forecasts = results_df['fc'].values
    horizon = 14
    
    mean_err, error_dist = err.test_errors_nrmse(train_df.values, test_sample, forecasts, horizon)
    return mean_err, error_dist

In [70]:
def read_loss_curve(path):
    return pd.read_pickle(f'../{path}/training_loss_grid_iteration')

def plot_loss(loss_array, models):

    fig = make_subplots(rows = len(loss_array), shared_xaxes=True)
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ), row = i+1, col=1)
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'), row = i+1, col=1)
        i +=1
        
    fig.update_xaxes(title="Epochs", row=i, col=1)
    fig.show()
    
def show_loss(path_list, model_names):
    loss_vals = []
    for path in path_list:
        loss_vals.append(read_loss_curve(path))
    plot_loss(loss_vals, model_names)

## Model 1 - Architecture

<img src="../images/New_images/model1.png">

In [53]:
path1 = 'combined_nn_results/refined_models/model1'
rmse1, dist1 = calculate_grid_error(path1)

In [54]:
rmse1

0.4689518104839537

## Model 2 - Architecture

<img src="../images/new_images/model2.png">

In [63]:
path2 = 'combined_nn_results/refined_models/model2'
rmse2, dist2 = calculate_grid_error(path2)

In [65]:
rmse2

0.4851028450736508

## Model 3 - Architecture

<img src="../images/new_images/model3.png">

In [66]:
# dense layer 14 output from each branch
path3 = 'combined_nn_results/refined_models/model3'
rmse3, dist3 = calculate_grid_error(path3)

FileNotFoundError: [Errno 2] No such file or directory: '../combined_nn_results/refined_models/model3/grid.csv'

In [27]:
rmse3

0.6145534915204655

In [28]:
# dense layer 40 output from each branch
path3_1 = 'combined_nn_results/new_models/model6'
rmse3_1, dist3_1 = calculate_grid_error(path3_1)

In [29]:
rmse3_1

0.6308694408841671

## Model 4 - Architecture 

<img src="../images/model4.png">

In [30]:
# dense layer 40 output from each branch
path4 = 'combined_nn_results/new_models/model7'
rmse4, dist4 = calculate_grid_error(path4)

In [31]:
rmse4

0.6468553468423276

## Model 5 - Architecture

In [32]:
path5 = 'combined_nn_results/new_models/model8'
rmse5, dist5 = calculate_grid_error(path5)

In [33]:
rmse5

1.26047984138004

## Model 6 - Activation Changed

In [34]:
path6 = 'combined_nn_results/new_models/model9'
rmse6, dist6 = calculate_grid_error(path6)

In [35]:
rmse6

0.42020243819593356

In [36]:
dist6[14]

0.5366764211387419

In [37]:
path7 = 'combined_nn_results/new_models/model10'
rmse7, dist7 = calculate_grid_error(path7)

In [38]:
rmse7

0.41230646129093734

## Loss Curves

In [67]:
show_loss([path1, path2], ['model 1', 'model 2'])

## Plot the forecasts

In [56]:
def change_df(dataframe):
    dataframe.index = pd.to_datetime(dataframe.reset_index()['date_str'])
    dataframe = dataframe.resample("1H").mean()
    return dataframe

In [57]:
arima = pd.read_csv(f'../benchmark_results/arima/grid.csv', index_col=[0])
arima = change_df(arima)

In [58]:
tbats = pd.read_csv(f'../benchmark_results/tbats/grid.csv', index_col=[0])
tbats = change_df(tbats)

In [59]:
model1_fc = pd.read_csv(f'../{path1}/grid.csv', index_col=[0])
model1_fc = change_df(model1_fc)

In [43]:
model2_fc = pd.read_csv(f'../{path2}/grid.csv', index_col=[0])
model2_fc = change_df(model2_fc)

In [60]:
path_cnn = 'cnn_results/tcn_new/dilation_2/final_results'

cnn_uni = pd.read_csv(f'../{path_cnn}/grid.csv', index_col=[0])
cnn_uni = change_df(cnn_uni)

In [62]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = model1_fc.index, y = model1_fc['power'], name = 'target generation'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = cnn_uni['average_fc'], name = 'Grid level time series only forecasts'))
fig.add_trace(go.Scatter(x = model1_fc.index, y = model1_fc['fc'], name = 'Combine Grid+PC NN forecasts'))


fig.update_yaxes(title = "Generation (kW)")
fig.show()
fig.write_html('forecasts.html')

## Let's have a look at the data scaling

In [68]:
from sklearn.preprocessing import StandardScaler
def create_window_data(file_index, lookback):
    filename = const.TS[file_index]

    horizon = 14  # day ahead forecast
    data = pd.read_csv(f'../ts_data/{filename}.csv', index_col=[0])
    print(filename)
    # 14 hours in to 7 days
    look_back = 14 * lookback

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)

    scaler = StandardScaler()
    scaler.fit(train.values)
    train_array = scaler.transform(train.values)
    val_array = scaler.transform(val.values)
    test_array = scaler.transform(test.values)

    train_df = pd.DataFrame(train_array, columns=data.columns)
    val_df = pd.DataFrame(val_array, columns=data.columns)
    test_df = pd.DataFrame(test_array, columns=data.columns)
    
    return test_df

In [69]:
lookback = 7

window_grid = create_window_data(0, lookback)
window_pc_6010 = create_window_data(7, lookback)
window_pc_6014 = create_window_data(8, lookback)
window_pc_6011 = create_window_data(9, lookback)
window_pc_6280 = create_window_data(10, lookback)
window_pc_6281 = create_window_data(11, lookback)
window_pc_6284 = create_window_data(12, lookback)

grid
6010
6014
6011
6280
6281
6284


In [70]:
# window_pc_6010

In [71]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = window_grid['power'], name= 'grid_data'))
fig.add_trace(go.Scatter(y = window_pc_6010['power'], name = 'pc_6010_data'))
fig.add_trace(go.Scatter(y = window_pc_6014['power'], name = 'pc_6014_data'))
fig.add_trace(go.Scatter(y = window_pc_6011['power'], name = 'pc_6011_data'))
fig.add_trace(go.Scatter(y = window_pc_6280['power'], name = 'pc_6280_data'))
fig.add_trace(go.Scatter(y = window_pc_6281['power'], name = 'pc_6281_data'))
fig.add_trace(go.Scatter(y = window_pc_6284['power'], name = 'pc_6284_data'))